# TRT-35: Harmony STAC for Analysis Ready Data

### Notebook goal

Test a basic STAC catalog workflow from:
  *  Harmony 
  * CMR-STAC 
  * VEDA endpoint
  * External (e.g. Planetary Computer) STAC catalog?

to determine:
1. Interoperability with common stac libraries
2. Feature parity vs gaps in functionality for Harmony
3. Utility as input into VEDA workflow


## 1. Determine dataset/use case 
Resources: 
* https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html for CMR-STAC example
* https://nasa-openscapes.github.io/earthdata-cloud-cookbook/examples/LPDAAC/Find_and_Access_HLS_PointBuffer.html for another CMR-STAC example
* https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb for Harmony STAC example
* https://nasa-impact.github.io/veda-docs/ for VEDA examples
* https://staging-stac.delta-backend.com/ : VEDA STAC API

We can use HLS example for CMR-STAC and a COG output example from Harmony to control for file format.

## 2. CMR-STAC Example
Copying from LP DAAC, VEDA, and Harmony tutorials above

What are the common STAC library imports across the three?

`from pystac_client import Client` is common across CMR-STAC and VEDA

**Questions for stakholders:** 
1. What is best practice when it comes to using `pystac_client`? 
2. Is Harmony's use of `pystac` incorrect? From 2i2c Hub I get an error importing STAC_IO so it is commented out below
3. Should we support both `pystac_client` and `intake`?

In [2]:
# Imports needed from LP DAAC tutorial:

from shapely.geometry import Point
import json
import geopandas
import rasterio as rio
import rioxarray
import os
from pystac_client import Client 
import hvplot.xarray

# Imports needed from VEDA tutorial (https://nasa-impact.github.io/veda-docs/example-notebooks/open-and-plot.html):
from pystac_client import Client
import xarray as xr

import hvplot.xarray  # noqa

#Imports needed from Harmony
# Using PySTAC
from urllib.parse import urlparse
import requests
#from pystac import STAC_IO
from pystac import Catalog
# Using intake
import intake


ImportError: cannot import name 'ParserError' from 'dateutil.parser' (/srv/conda/envs/notebook/lib/python3.9/site-packages/dateutil/parser/__init__.py)

### Set geometry for spatial search:

In [22]:
field = geopandas.read_file('./data/ne_w_agfields.geojson')
roi = json.loads(field.to_json())['features'][0]['geometry']
roi

{'type': 'Polygon',
 'coordinates': [[[-101.67271614074707, 41.04754380304359],
   [-101.65344715118408, 41.04754380304359],
   [-101.65344715118408, 41.06213891056728],
   [-101.67271614074707, 41.06213891056728],
   [-101.67271614074707, 41.04754380304359]]]}

In [30]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
catalog = Client.open(f'{STAC_URL}/LPCLOUD/')

#New version of CMR (CMR-1.217.0-r23.1.1) deployed 2/22 now requires concept-id, not shortname. I still can't get this to work using collection-id however:
# HLSL30.v2.0 --> C2021957657-LPCLOUD
# HLSS30.v2.0 --> C2021957295-LPCLOUD
search = catalog.search(
    collections = "C2021957657-LPCLOUD",
#    collections = ["HLSL30.v2.0"],
#    intersects = roi,
    datetime = '2021-05/2021-08'
)               
search.matched()

2001869

In [31]:
items = search.get_all_items()
print(items[0])
items[0].assets
# data_url = items[0].assets['B04'].href
# data_url

<Item id=G2484151012-LPCLOUD>


{'data': <Asset href=https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MCD12C1.061/MCD12C1.A2021001.061.2022217040006/MCD12C1.A2021001.061.2022217040006.hdf>,
 'provider_metadata': <Asset href=https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MCD12C1.061/MCD12C1.A2021001.061.2022217040006/MCD12C1.A2021001.061.2022217040006.cmr.xml>,
 'metadata': <Asset href=https://cmr.earthdata.nasa.gov/search/concepts/G2484151012-LPCLOUD.json>}

In [ ]:
# #We can do this with the latest version of earthaccess so that we don't have to use the GDAL driver code below. But, this needs to be tested first:  

# import earthaccess

# da = xr.open_mfdataset(earthaccess.open([data_url]), engine="rasterio")
# da = da.squeeze("band", drop=True)
# da

In [ ]:
rio_env = rio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

In [ ]:
da = rioxarray.open_rasterio(data_url)
da = da.squeeze('band', drop=True)
da

In [ ]:
da.hvplot.image(x='x', y='y', cmap='fire', rasterize=True, width=800, height=600, colorbar=True)    # colormaps -> https://holoviews.org/user_guide/Colormaps.html

## 3. VEDA Example


In [35]:
STAC_API_URL = "https://staging-stac.delta-backend.com/"
collection = "social-vulnerability-index-overall-nopop"

In [37]:
search = Client.open(STAC_API_URL).search(collections=[collection])
items = list(search.items())
items[0]

ID: svi_2018_tract_overall_wgs84_nopop_cog
"Bounding Box: [-178.23333334, 18.908332897999998, -66.958333785, 71.383332688]"
"proj:bbox: [-178.23333334, 18.908332897999998, -66.958333785, 71.383332688]"
proj:epsg: 4326.0
"proj:shape: [6297.0, 13353.0]"
end_datetime: 2018-12-31T00:00:00
"proj:geometry: {'type': 'Polygon', 'coordinates': [[[-178.23333334, 18.908332897999998], [-66.958333785, 18.908332897999998], [-66.958333785, 71.383332688], [-178.23333334, 71.383332688], [-178.23333334, 18.908332897999998]]]}"
"proj:transform: [0.00833333330000749, 0.0, -178.23333334, 0.0, -0.00833333329998412, 71.383332688, 0.0, 0.0, 1.0]"
start_datetime: 2018-01-01T00:00:00
"stac_extensions: ['https://stac-extensions.github.io/projection/v1.0.0/schema.json', 'https://stac-extensions.github.io/raster/v1.1.0/schema.json']"
https://stac-extensions.github.io/projection/v1.0.0/schema.json


In [39]:
s3_link = items[0].assets['cog_default'].href
print(s3_link)

# #I dont have access yet to the data:
# da = xr.open_dataset(s3_link, engine="rasterio")
# da = da.squeeze("band", drop=True)
# da

s3://veda-data-store-staging/social-vulnerability-index-overall-nopop/svi_2018_tract_overall_wgs84_nopop_cog.tif


## 4. Harmony Example

In [41]:
import sys
!{sys.executable} -m pip install -U harmony-py

  Using cached harmony_py-0.4.4-py3-none-any.whl (23 kB)
  Using cached python_dateutil-2.7.5-py2.py3-none-any.whl (225 kB)
  Using cached sphinxcontrib_napoleon-0.7-py2.py3-none-any.whl (17 kB)
  Using cached python_dotenv-0.21.1-py3-none-any.whl (19 kB)
  Using cached curlify-2.2.1-py3-none-any.whl
  Using cached progressbar2-3.55.0-py2.py3-none-any.whl (26 kB)
  Using cached python_utils-3.5.2-py2.py3-none-any.whl (24 kB)
  Using cached pockets-0.9.1-py2.py3-none-any.whl (26 kB)
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:
      Successfully uninstalled python-dateutil-2.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-benedict 0.26.0 requires python-dateutil<3.0.0,>=2.8.0, but you have python-dateutil 2.7.5 which is incompatible.
pandas 1.5.3 requires python-

Now need to restart kernel

In [1]:
from harmony import BBox, Client, Collection, Request
from harmony.config import Environment
import datetime as dt

import intake

In [3]:
harmony_client = Client() # assumes .netrc usage, option 3 above

In [4]:
shapefile_path = './data/Big_Island_0005.zip' 

request = Request(
    collection=Collection(id='SENTINEL-1_INTERFEROGRAMS'),
    #spatial=BBox(-155.75, 19.26, -155.3, 19.94), # bounding box example that can be used as an alternative to shapefile input
    shape=shapefile_path,
    temporal={
        'start': dt.datetime(2020, 2, 24),
        'stop': dt.datetime(2020, 2, 25),
    },
    variables=['science/grids/data/unwrappedPhase'],
    format='image/tiff',
    max_results=2,
)

In [5]:
job_id = harmony_client.submit(request)
job_id

'111892c1-1f95-4be4-b278-031c41564a5c'

In [6]:
print(harmony_client.status(job_id))
harmony_client.wait_for_processing(job_id, show_progress=True)

{'status': 'running', 'message': 'CMR query identified 10 granules, but the request has been limited to process only the first 2 granules because you requested 2 maxResults.', 'progress': 0, 'created_at': datetime.datetime(2023, 2, 23, 1, 44, 3, 648000, tzinfo=tzlocal()), 'updated_at': datetime.datetime(2023, 2, 23, 1, 44, 3, 648000, tzinfo=tzlocal()), 'created_at_local': '2023-02-23T01:44:03+00:00', 'updated_at_local': '2023-02-23T01:44:03+00:00', 'data_expiration': datetime.datetime(2023, 3, 25, 1, 44, 3, 648000, tzinfo=tzlocal()), 'data_expiration_local': '2023-03-25T01:44:03+00:00', 'request': 'https://harmony.earthdata.nasa.gov/SENTINEL-1_INTERFEROGRAMS/ogc-api-coverages/1.0.0/collections/science%2Fgrids%2Fdata%2FunwrappedPhase/coverage/rangeset?forceAsync=true&subset=time(%222020-02-24T00%3A00%3A00%22%3A%222020-02-25T00%3A00%3A00%22)&format=image%2Ftiff&maxResults=2', 'num_input_granules': 2}


 [ Processing: 100% ] |###################################################| [|]


In [ ]:
# urls = harmony_client.result_urls(job_id)
# list(urls)

In [8]:
stac_catalog_url = harmony_client.stac_catalog_url(job_id)
stac_catalog_url

'https://harmony.earthdata.nasa.gov/stac/111892c1-1f95-4be4-b278-031c41564a5c/?linktype=https'

Before trying to use the same CMR-STAC or VEDA workflow above, we need to understand how this STAC URL compares to CMR-STAC as far as the STAC spec: According to https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html:

There are Four STAC Specifications:
*STAC Catalog (aka DAAC Archive)
*STAC Collection (aka Data Product)
*STAC Item (aka Granule)
*STAC API

**Given that the Harmony STAC URL contains items, does this mean that this is technically a STAC Collection URL?**

And according to:
https://pystac-client.readthedocs.io/en/stable/tutorials/pystac-client-introduction.html

"If the Catalog is an API, we have the ability to search for items based on spatio-temporal properties."

**So, we also need to confirm whether Harmony STAC endpoint is an API or not.** Based on the VEDA and CMR-STAC tutorials, those catalogs appear to be APIs as well. 



### Now try the above CMR-STAC workflow:

The commented line above gives me this error:

```ClientTypeError: Could not open Client (href=https://harmony.earthdata.nasa.gov/stac/111892c1-1f95-4be4-b278-031c41564a5c/?linktype=https), expected type=Catalog, found type=None```

In [12]:
from pystac_client import Client 
#catalog = Client.open(stac_catalog_url)


I had to modify the existing Harmony-py tutorial to update based on the current `pystac` documentation (commented line is the old code:

In [24]:
from urllib.parse import urlparse
import requests
from pystac.stac_io import StacIO
#from pystac import STAC_IO

def requests_read_method(uri):
    parsed = urlparse(uri)
    if parsed.hostname.startswith('harmony.'):
        return harmony_client.read_text(uri)
    else:
        return StacIO.default_read_text_method(uri)

StacIO.read_text_method = requests_read_method

The next code block from the Harmony-py tutorial throws this error:

```Exception: Could not read uri https://harmony.earthdata.nasa.gov/stac/111892c1-1f95-4be4-b278-031c41564a5c/?linktype=https

In [26]:
# from pystac import Catalog

# cat = Catalog.from_file(stac_catalog_url)

# print(cat.title)
# s3_links = []
# for item in cat.get_all_items():
#     print(item.datetime, [asset.href for asset in item.assets.values()])
#     s3_links.append([asset.href for asset in item.assets.values()])

This recent Pangeo discourse post may be a hint, as Tom A. mentions that this could be a difference between the API and a static catalog: https://discourse.pangeo.io/t/pystac-client-cannot-load-stac-catalog/3071

```import pystac
pystac.read_file("https://meeo-s5p.s3.amazonaws.com/catalog.json")```

I get an error when I try with the Harmony URL:

In [28]:
import pystac
#pystac.read_file(stac_catalog_url)